In [1]:
!pip install datasets
from datasets import load_dataset
dataset=load_dataset('Helsinki-NLP/opus-100','en-es')
train_dataset=dataset['train']
val_dataset=dataset['validation']
test_dataset=dataset['test']

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [2]:
import sys
sys.path.append('/content/drive/MyDrive/ColabNotebooks')
from transformer import Transformer

In [3]:
START_TOKEN = '<SOE>'
PADDING_TOKEN = '<PAD>'
END_TOKEN = '<EOE>'

english_vocab = [START_TOKEN, ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/',
                        '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
                        ':', '<', '=', '>', '?', '@',
                        '[', '\\', ']', '^', '_', '`',
                        'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l',
                        'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x',
                        'y', 'z',
                        '{', '|', '}', '~', PADDING_TOKEN, END_TOKEN]
spanish_vocab = [START_TOKEN, ' ','¡', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/',
                        '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
                        ':', '<', '=', '>', '¿','?', '@',
                        '[', '\\', ']', '^', '_', '`',
                        'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l',
                        'm', 'n','ñ', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x',
                        'y', 'z','á','é','í','ó','ú',
                        '{', '|', '}', '~', PADDING_TOKEN, END_TOKEN]

ind_to_esp = {k:v for k,v in enumerate(spanish_vocab)}
esp_to_ind = {v:k for k,v in enumerate(spanish_vocab)}
ind_to_eng = {k:v for k,v in enumerate(english_vocab)}
eng_to_ind = {v:k for k,v in enumerate(english_vocab)}

In [4]:
import torch
import numpy as np
NEG_INFTY = -1e9

def create_masks(eng_batch, esp_batch):
    num_sentences = len(eng_batch)
    look_ahead_mask = torch.full([max_sequence_length, max_sequence_length] , True)
    look_ahead_mask = torch.triu(look_ahead_mask, diagonal=1)
    encoder_padding_mask = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)
    decoder_padding_mask_self_attention = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)
    decoder_padding_mask_cross_attention = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)

    for idx in range(num_sentences):
      eng_sentence_length, esp_sentence_length = len(eng_batch[idx]), len(esp_batch[idx])
      eng_chars_to_padding_mask = np.arange(eng_sentence_length + 1, max_sequence_length)
      esp_chars_to_padding_mask = np.arange(esp_sentence_length + 1, max_sequence_length)
      encoder_padding_mask[idx, :, eng_chars_to_padding_mask] = True
      encoder_padding_mask[idx, eng_chars_to_padding_mask, :] = True
      decoder_padding_mask_self_attention[idx, :, esp_chars_to_padding_mask] = True
      decoder_padding_mask_self_attention[idx, esp_chars_to_padding_mask, :] = True
      decoder_padding_mask_cross_attention[idx, :, eng_chars_to_padding_mask] = True
      decoder_padding_mask_cross_attention[idx, esp_chars_to_padding_mask, :] = True

    encoder_self_attention_mask = torch.where(encoder_padding_mask, NEG_INFTY, 0)
    decoder_self_attention_mask =  torch.where(look_ahead_mask + decoder_padding_mask_self_attention, NEG_INFTY, 0)
    decoder_cross_attention_mask = torch.where(decoder_padding_mask_cross_attention, NEG_INFTY, 0)
    return encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask

In [5]:
def translate(transformer,eng_sentence):
  device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
  transformer.eval()
  eng_sentence = (eng_sentence,)
  esp_sentence = ("",)
  for word_counter in range(max_sequence_length):
    encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask= create_masks(eng_sentence, esp_sentence)
    predictions = transformer(eng_sentence,
                              esp_sentence,
                              encoder_self_attention_mask.to(device),
                              decoder_self_attention_mask.to(device),
                              decoder_cross_attention_mask.to(device),
                              enc_start_token=False,
                              enc_end_token=False,
                              dec_start_token=True,
                              dec_end_token=False)
    next_token_prob_distribution = predictions[0][word_counter]
    next_token_index = torch.argmax(next_token_prob_distribution).item()
    next_token = ind_to_esp[next_token_index]
    esp_sentence = (esp_sentence[0] + next_token, )
    if next_token == END_TOKEN:
      break
  return esp_sentence[0]

In [6]:
d_model = 512
batch_size = 50
ffn_hidden = 2048
num_heads_1 = 16
num_heads_2 = 8
num_heads_3 = 8
drop_prob = 0.1
num_layers_1 = 1
num_layers_2 = 2
num_layers_3 = 4
max_sequence_length = 300
esp_vocab_size = len(spanish_vocab)

transformer_base=Transformer(d_model,
                          ffn_hidden,
                          num_heads_2,
                          drop_prob,
                          num_layers_1,
                          max_sequence_length,
                          esp_vocab_size,
                          eng_to_ind,
                          esp_to_ind,
                          START_TOKEN,
                          END_TOKEN,
                          PADDING_TOKEN)
transformer_base.load_state_dict(torch.load('/content/drive/MyDrive/ColabNotebooks/Models/transformer_base',map_location=torch.device('cuda')))
transformer_base.to(torch.device('cuda'))

transformer1=Transformer(d_model,
                          ffn_hidden,
                          num_heads_1,
                          drop_prob,
                          num_layers_1,
                          max_sequence_length,
                          esp_vocab_size,
                          eng_to_ind,
                          esp_to_ind,
                          START_TOKEN,
                          END_TOKEN,
                          PADDING_TOKEN)
transformer1.load_state_dict(torch.load('./drive/MyDrive/ColabNotebooks/Models/transformer1',map_location=torch.device('cuda')))
transformer1.to(torch.device('cuda'))

transformer2=Transformer(d_model,
                          ffn_hidden,
                          num_heads_2,
                          drop_prob,
                          num_layers_2,
                          max_sequence_length,
                          esp_vocab_size,
                          eng_to_ind,
                          esp_to_ind,
                          START_TOKEN,
                          END_TOKEN,
                          PADDING_TOKEN)
transformer2.load_state_dict(torch.load('./drive/MyDrive/ColabNotebooks/Models/transformer2',map_location=torch.device('cuda')))
transformer2.to(torch.device('cuda'))

transformer_final = Transformer(d_model,
                          ffn_hidden,
                          num_heads_3,
                          drop_prob,
                          num_layers_3,
                          max_sequence_length,
                          esp_vocab_size,
                          eng_to_ind,
                          esp_to_ind,
                          START_TOKEN,
                          END_TOKEN,
                          PADDING_TOKEN)
transformer_final.load_state_dict(torch.load('./drive/MyDrive/ColabNotebooks/Models/transformer_final.pth.tar',map_location=torch.device('cuda')))
transformer_final.to(torch.device('cuda'))

Transformer(
  (encoder): Encoder(
    (sentence_embedding): SentenceEmbedding(
      (embedding): Embedding(71, 512)
      (position_encoder): PositionalEncoding()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (layers): SequentialEncoder(
      (0): EncoderLayer(
        (attention): MultiHeadAttention(
          (qkv_layer): Linear(in_features=512, out_features=1536, bias=True)
          (linear_layer): Linear(in_features=512, out_features=512, bias=True)
        )
        (norm1): LayerNormalization()
        (dropout1): Dropout(p=0.1, inplace=False)
        (ffn): PositionwiseFeedForward(
          (linear1): Linear(in_features=512, out_features=2048, bias=True)
          (linear2): Linear(in_features=2048, out_features=512, bias=True)
          (relu): ReLU()
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (norm2): LayerNormalization()
        (dropout2): Dropout(p=0.1, inplace=False)
      )
      (1): EncoderLayer(
        (attention): MultiHeadAt

In [10]:
def is_valid_tokens(sentence, vocab):
    for token in list(set(sentence)):
        if token not in vocab:
          return False
    return True

def is_valid_length(sentence, max_sequence_length):
    return len(list(sentence)) < (max_sequence_length - 1)

In [ ]:
errores=0
original_sentence=[]
spanish_translation=[]
prediction_transformer_base=[]
prediction_transformer1=[]
prediction_transformer2=[]
prediction_transformer3=[]
prediction_transformer_final=[]
for i in range(len(test_dataset)):
  row=test_dataset[i]['translation']
  try:
    sentence=row['en'].lower()
    if is_valid_tokens(sentence,english_vocab) and is_valid_length(sentence,300):
      original_sentence.append(sentence)
      traduccion_base=translate(transformer_base,sentence)
      traduccion1=translate(transformer1,sentence)
      traduccion2=translate(transformer2,sentence)
      # traduccion3=translate(transformer3,sentence)
      traduccion_final=translate(transformer_final,sentence)
      print(f'Sentence {i}: ',row['en'].lower())
      spanish_translation.append(row['es'].lower())
      print('Original Translation: ',row['es'].lower())
      prediction_transformer_base.append(traduccion_base)
      prediction_transformer1.append(traduccion1)
      prediction_transformer2.append(traduccion2)
      # prediction_transformer3.append(traduccion3)
      prediction_transformer_final.append(traduccion_final)
      print('Transformer Final: ',traduccion_final)
      print('-----------------------------------------------------')
      if i%100==0:
        print(f'Iteration {i}')
      i+=1
  except:
    errores+=1

In [7]:
#Los resultados de las traducciones están almacenados en el fichero translation_arrays
import sys
sys.path.append('/content/drive/MyDrive/ColabNotebooks')
import translation_arrays

spanish_translation=[[item] for item in translation_arrays.spanish_translation]

In [8]:
!pip install sacrebleu
import sacrebleu


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 3.7 MB/s eta 0:00:00


In [9]:
bleu_transformer_base=sacrebleu.corpus_bleu(translation_arrays.prediction_transformer_base[1000:],spanish_translation[:1000])
print("BLEU SCORE TRANSFORMER BASE: ", bleu_transformer_base.score)
bleu_transformer1=sacrebleu.corpus_bleu(translation_arrays.prediction_transformer1[1000:],spanish_translation[1000:])
print("BLEU SCORE TRANSFORMER 1: ", bleu_transformer1.score)
bleu_transformer2=sacrebleu.corpus_bleu(translation_arrays.prediction_transformer2[1000:],spanish_translation[1000:])
print("BLEU SCORE TRANSFORMER 2: ", bleu_transformer2.score)
# bleu_transformer3=sacrebleu.corpus_bleu(translation_arrays.prediction_transformer3[1000:],spanish_translation[1000:])
# print("BLEU SCORE TRANSFORMER 3: ", bleu_transformer3.score)
bleu_transformer_final=sacrebleu.corpus_bleu(translation_arrays.prediction_transformer_final[1000:],spanish_translation[1000:])
print("BLEU SCORE TRANSFORMER FINAL: ", bleu_transformer_final.score)

BLEU SCORE TRANSFORMER BASE:  8.475426399505565
BLEU SCORE TRANSFORMER 1:  7.77536842701581
BLEU SCORE TRANSFORMER 2:  11.382946616278247
BLEU SCORE TRANSFORMER FINAL:  28.175950490399515
